In [44]:
import sys
import time
import logging
import os
import opcua

In [38]:
# url = "opc.tcp://130.159.75.138:48050"
# url = "opc.tcp://CX-696B93:4840"
url = "opc.tcp://130.159.75.138:4840"

In [39]:
client = opcua.Client(url = url)
client.set_user(username = 'Administrator')
client.set_password(pwd = '*500TPress*')
client.connect()

In [40]:
p = client.get_node("ns=4;s=GVL_Analog_Engineering_Units.lrEL3202_1_Ambient_Temperature_Front")
var = p.get_data_value()
var.Value.Value

19.8

In [41]:
nodeid_list = ["ns=4;s=GVL_OPTO22.Press_Position",
               "ns=4;s=GVL_OPTO22.Main_Cylinder_Pressure",
               "ns=4;s=GVL_OPTO22.Bottom_Die_Temperature",
               "ns=4;s=GVL_OPTO22.Top_Die_Temperature",
               "ns=4;s=GVL_OPTO22.Bottom_Die_Heater_Power_Demand",
               "ns=4;s=GVL_OPTO22.Top_Die_Heater_Power_Demand",
               "ns=4;s=GVL_OPTO22.Pump_Pressure_Demand",
               "ns=4;s=GVL_OPTO22.Pump_Flow_Demand",
               "ns=4;s=GVL_Analog_Engineering_Units.lrEL3202_1_Ambient_Temperature_Front",
               "ns=4;s=GVL_Analog_Engineering_Units.lrEL3202_2_Ambient_Temperature_Back",
               "ns=4;s=GVL_Analog_Engineering_Units.nEL3318_1_Engineering_Unit",
               "ns=4;s=GVL_Analog_Engineering_Units.nEL3318_2_Engineering_Unit",
               "ns=4;s=GVL_Analog_Engineering_Units.nEL3318_3_Engineering_Unit",
               "ns=4;s=GVL_Analog_Engineering_Units.nEL3318_4_Engineering_Unit",
               "ns=4;s=GVL_Analog_Engineering_Units.nEL3318_5_Engineering_Unit",
               "ns=4;s=GVL_Analog_Engineering_Units.nEL3318_6_Engineering_Unit",
               "ns=4;s=GVL_Analog_Engineering_Units.nEL3318_7_Engineering_Unit",
               "ns=4;s=GVL_Analog_Engineering_Units.nEL3318_8_Engineering_Unit",
               "ns=4;s=GVL_Analog_Engineering_Units.rTotal_KW_MainsIncomer"]

In [42]:
nodeid_ua = [client.get_node(nodeid) for nodeid in nodeid_list]

In [43]:
node_vals = client.get_values(nodeid_ua)
node_vals

[147.39999389648438,
 -0.29998779296875,
 20.557220458984375,
 20.944107055664062,
 0.0,
 0.0,
 0.0,
 0.0,
 19.8,
 19.5,
 20.799999237060547,
 1372.0,
 1372.0,
 1372.0,
 1372.0,
 1372.0,
 1372.0,
 24.200000762939453,
 58.45683670043945]

In [30]:
while i < 500
        start = time.time()
        print(i)
        stop = time.time()
        print(stop-start)
        i+

[147.39999389648438,
 -0.33001708984375,
 20.557220458984375,
 20.944107055664062,
 0.0,
 0.0,
 0.0,
 0.0,
 19.8,
 19.4,
 20.600000381469727,
 1372.0,
 1372.0,
 1372.0,
 1372.0,
 1372.0,
 1372.0,
 24.200000762939453,
 58.45302963256836]

In [22]:



temp = opcua.Node(client, "ns=4;s=GVL_Analog_Engineering_Units.lrEL3202_1_Ambient_Temperature_Front")

m = client.get_values(temp)

TypeError: 'Node' object is not iterable

In [15]:
root = client.get_root_node()
root

Node(TwoByteNodeId(i=84))

In [16]:
root = client.get_root_node()
print("Root node is: {:s} ".format(str(root)))

def browse_recursive(node):
    for childId in node.get_children():
        ch = client.get_node(childId)
        print(ch.get_node_class())
        if ch.get_node_class() == opcua.ua.NodeClass.Object:
            browse_recursive(ch)
        elif ch.get_node_class() == opcua.ua.NodeClass.Variable:
            try:
                print("{bn} has value {val}".format(
                    bn=ch.get_browse_name(),
                    val=str(ch.get_value()))
                )
            except ua.uaerrors._auto.BadWaitingForInitialData:
                pass

browse_recursive(root)

Root node is: i=84 
NodeClass.Object
NodeClass.Object
NodeClass.Object
NodeClass.Variable
QualifiedName(0:ServerArray) has value ['urn:CX-696B93:BeckhoffAutomation:TcOpcUaServer:1']
NodeClass.Variable
QualifiedName(0:NamespaceArray) has value ['http://opcfoundation.org/UA/', 'urn:CX-696B93:BeckhoffAutomation:TcOpcUaServer:1', 'http://opcfoundation.org/UA/DI/', 'http://PLCopen.org/OpcUa/IEC61131-3/', 'urn:BeckhoffAutomation:Ua:PLC1', 'urn:BeckhoffAutomation:Ua:Types:GlobalTypes', 'urn:BeckhoffAutomation:Ua:CM_Unit', 'urn:BeckhoffAutomation:Ua:HA', 'urn:BeckhoffAutomation:Ua:AC', 'http://Beckhoff.com/TwinCAT/TF6100/Server/Configuration']
NodeClass.Variable
QualifiedName(0:ServerStatus) has value ServerStatusDataType(StartTime:{self.StartTime}, CurrentTime:{self.CurrentTime}, State:{self.State}, BuildInfo:{self.BuildInfo}, SecondsTillShutdown:{self.SecondsTillShutdown}, ShutdownReason:{self.ShutdownReason})
NodeClass.Variable
QualifiedName(0:ServiceLevel) has value 255
NodeClass.Object
No

BadSecurityModeInsufficient: "The operation is not permitted over the current secure channel."(BadSecurityModeInsufficient)

In [ ]:
objects = client.get_objects_node()
childs = objects.get_children()